In [1]:
def read_grammar(file_path):
    grammar = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line:
                lhs, rhs = line.split('=')
                lhs = lhs.strip()
                rhs = rhs.strip().split('|')
                grammar[lhs] = [rule.strip().split() for rule in rhs]
    return grammar

def compute_first_sets(grammar):
    first = {non_terminal: set() for non_terminal in grammar}

    def first_of(symbol):
        if symbol not in grammar:  # Terminal symbol
            return {symbol}
        if not first[symbol]:  # Non-terminal, not yet computed
            for production in grammar[symbol]:
                for sym in production:
                    sym_first = first_of(sym)
                    first[symbol].update(sym_first)
                    
        return first[symbol]

    for non_terminal in grammar:
        first_of(non_terminal)
    return first

def compute_follow_sets(grammar, start_symbol):
    follow = {non_terminal: set() for non_terminal in grammar}
    follow[start_symbol].add('$')

    def follow_of(non_terminal):
        for lhs in grammar:
            for production in grammar[lhs]:
                for i, sym in enumerate(production):
                    if sym == non_terminal:
                        if i + 1 < len(production):
                            next_symbol = production[i + 1]
                            if next_symbol in grammar:
                                follow[non_terminal].update(first[next_symbol] - {'ɛ'})
                                if 'ɛ' in first[next_symbol]:
                                    follow[non_terminal].update(follow_of(lhs))
                            else:
                                follow[non_terminal].add(next_symbol)
                        else:
                            if lhs != non_terminal:
                                follow[non_terminal].update(follow_of(lhs))
        return follow[non_terminal]

    for non_terminal in grammar:
        follow_of(non_terminal)
    return follow

# Example grammar
grammar = {
    'S': [['X', 'a']],
    'X': [['a', 'X'], ['b', 'X'], ['ɛ']]
}

# Compute FIRST sets
first = compute_first_sets(grammar)

# Compute FOLLOW sets
start_symbol = next(iter(grammar))
follow = compute_follow_sets(grammar, start_symbol)

# Print FIRST sets in the desired format
print("FIRST sets:")
for non_terminal in first:
    print(f"{non_terminal} = {first[non_terminal]}")

# Print FOLLOW sets in the desired format
print("\nFOLLOW sets:")
for non_terminal in follow:
    print(f"{non_terminal} = {follow[non_terminal]}")


FIRST sets:
S = {'a', 'ɛ', 'b'}
X = {'a', 'ɛ', 'b'}

FOLLOW sets:
S = {'$'}
X = {'a'}
